# Capitulo 02 Integracao Duckdb Arrow

Notebook gerado automaticamente a partir do código fonte python.


In [ ]:
# -*- coding: utf-8 -*-
"""
Capítulo 02: Integração DuckDB + Arrow
Curso: Apache Arrow + DuckDB

Tópicos abordados:
- Modos de integração
- Conversões bidirecionais
- Query em Arrow Tables
- Schemas e metadados
- Error handling
"""

import sys
import io
import pyarrow as pa
import duckdb
import pandas as pd
import numpy as np

# Configuração para saída correta no Windows
sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

print("="*60)
print(f"CAPÍTULO 02: INTEGRAÇÃO DUCKDB + ARROW")
print("="*60)

# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    data = pa.table({
        'id': range(1000),
        'valor': np.random.randn(1000),
        'categoria': np.random.choice(['A', 'B', 'C'], 1000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()


# ==========================================
# Tópico 1: Modos de integração
# ==========================================
print(f"\n--- {'Modos de integração'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Modos de integração
# Dica: Use duckdb.query() ou funcoes do pyarrow

# ==========================================
# Tópico 2: Conversões bidirecionais
# ==========================================
print(f"\n--- {'Conversões bidirecionais'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Conversões bidirecionais
# Dica: Use duckdb.query() ou funcoes do pyarrow

# ==========================================
# Tópico 3: Query em Arrow Tables
# ==========================================
print(f"\n--- {'Query em Arrow Tables'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Query em Arrow Tables
# Dica: Use duckdb.query() ou funcoes do pyarrow

# ==========================================
# Tópico 4: Schemas e metadados
# ==========================================
print(f"\n--- {'Schemas e metadados'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Schemas e metadados
# Dica: Use duckdb.query() ou funcoes do pyarrow

# ==========================================
# Tópico 5: Error handling
# ==========================================
print(f"\n--- {'Error handling'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Error handling
# Dica: Use duckdb.query() ou funcoes do pyarrow


print("\n" + "="*60)
print(f"Fim do Capítulo 02")
print("="*60)
